<a href="https://colab.research.google.com/github/Nikita-devel/DS_Hw7/blob/main/DS_Hw7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install scikit-surprise
import pandas as pd
from surprise import SVD, SVDpp, NMF
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate

# Завантаження датасету movielens
data = Dataset.load_builtin('ml-100k')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163011 sha256=cf7e551808d90ffa5846e0aa28599113f7a15697ec7749a3fafb209e775a437f
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise
Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [5]:
df = pd.DataFrame(data.raw_ratings, columns=["user_id", "item_id", "rating", "timestamp"])
print(df.head())

  user_id item_id  rating  timestamp
0     196     242     3.0  881250949
1     186     302     3.0  891717742
2      22     377     1.0  878887116
3     244      51     2.0  880606923
4     166     346     1.0  886397596


In [6]:
svd_model = SVD()
svdpp_model = SVDpp()
nmf_model = NMF()

# Крос-валідація
cross_validate(svd_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
cross_validate(svdpp_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
cross_validate(nmf_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9389  0.9343  0.9445  0.9266  0.9383  0.9365  0.0059  
MAE (testset)     0.7413  0.7359  0.7458  0.7302  0.7369  0.7380  0.0052  
Fit time          1.39    1.38    1.50    2.61    2.88    1.95    0.65    
Test time         0.11    0.18    0.22    0.76    0.40    0.33    0.23    
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9193  0.9164  0.9228  0.9203  0.9188  0.9195  0.0021  
MAE (testset)     0.7192  0.7167  0.7235  0.7243  0.7201  0.7207  0.0028  
Fit time          26.57   29.73   26.71   26.54   26.54   27.22   1.26    
Test time         4.24    4.08    5.06    3.91    5.15    4.49    0.52    
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (tests

{'test_rmse': array([0.96164533, 0.96991303, 0.96505567, 0.96532858, 0.96489472]),
 'test_mae': array([0.75471961, 0.76316317, 0.75838656, 0.75799279, 0.75783454]),
 'fit_time': (1.7530038356781006,
  1.7934653759002686,
  1.8251404762268066,
  2.2057769298553467,
  2.181133508682251),
 'test_time': (0.09148716926574707,
  0.20727157592773438,
  0.0924828052520752,
  0.15981841087341309,
  0.2699766159057617)}

In [7]:
from surprise.model_selection import GridSearchCV
param_grid = {'n_epochs': [5, 10, 20], 'lr_all': [0.002, 0.005, 0.01],
              'reg_all': [0.4, 0.6, 0.8]}

# Використання GridSearchCV для пошуку оптимальних параметрів
grid_search = GridSearchCV(SVD, param_grid, measures=['RMSE'], cv=5)
grid_search.fit(data)

In [8]:
best_params = grid_search.best_params['rmse']
print("Best Parameters:", best_params)

Best Parameters: {'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.4}


In [9]:
final_svd_model = SVD(n_epochs=best_params['n_epochs'], lr_all=best_params['lr_all'], reg_all=best_params['reg_all'])
trainset = data.build_full_trainset()
final_svd_model.fit(trainset)

In [10]:
user_id = '42'
all_predictions = final_svd_model.test(trainset.build_anti_testset(fill=None))

# Виведення всіх прогнозів для користувача
for pred in all_predictions:
    print(f"Item ID: {pred.iid}, Prediction: {pred.est}")
#Краще не запускати...

Streaming output truncated to the last 5000 lines.
Item ID: 580, Prediction: 2.599648188533218
Item ID: 1041, Prediction: 2.42084520364851
Item ID: 424, Prediction: 1.2326084015154484
Item ID: 429, Prediction: 2.968839070807173
Item ID: 868, Prediction: 2.9651780293297123
Item ID: 1161, Prediction: 2.3729265075223234
Item ID: 166, Prediction: 3.0749237046391094
Item ID: 1162, Prediction: 1.7663979530648146
Item ID: 1019, Prediction: 3.056001342487283
Item ID: 908, Prediction: 2.0278158330718306
Item ID: 1160, Prediction: 2.7242362866894836
Item ID: 676, Prediction: 2.5476403625075226
Item ID: 1012, Prediction: 2.6632042798803175
Item ID: 1209, Prediction: 2.040442108049352
Item ID: 693, Prediction: 2.7909522561908036
Item ID: 282, Prediction: 2.779562741333127
Item ID: 253, Prediction: 2.7907046193860876
Item ID: 1350, Prediction: 2.306354162674308
Item ID: 363, Prediction: 2.19400980254853
Item ID: 300, Prediction: 2.747807021003317
Item ID: 1346, Prediction: 2.3337335431637047
Item I

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
Item ID: 644, Prediction: 3.75136891131592
Item ID: 982, Prediction: 2.9842354482812206
Item ID: 491, Prediction: 3.656961612315454
Item ID: 511, Prediction: 4.006476340211493
Item ID: 397, Prediction: 3.0130736929893467
Item ID: 162, Prediction: 3.503097342698059
Item ID: 513, Prediction: 4.086711463035704
Item ID: 375, Prediction: 2.4362285811972226
Item ID: 38, Prediction: 3.1635654269190447
Item ID: 418, Prediction: 3.541018943884433
Item ID: 858, Prediction: 2.7161106398100263
Item ID: 131, Prediction: 3.67749000830164
Item ID: 725, Prediction: 3.043326682004265
Item ID: 72, Prediction: 3.264730524278806
Item ID: 663, Prediction: 3.7745967985426594
Item ID: 1133, Prediction: 3.095888994515125
Item ID: 125, Prediction: 3.5442749777821656
Item ID: 1248, Prediction: 3.174343989763057
Item ID: 105, Prediction: 2.7576031665941
Item ID: 73, Prediction: 3.4929019660823326
Item ID: 239, Prediction: 3.5700078760004583
Item ID: 860, Predict

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
Item ID: 1070, Prediction: 3.5959217084922135
Item ID: 449, Prediction: 2.96707763751918
Item ID: 948, Prediction: 2.399101114732269
Item ID: 509, Prediction: 3.554405779675801
Item ID: 39, Prediction: 3.1668496336450493
Item ID: 1100, Prediction: 3.0134874781743477
Item ID: 660, Prediction: 3.5056508702273956
Item ID: 1022, Prediction: 3.3167667627266346
Item ID: 120, Prediction: 2.4230562291538353
Item ID: 780, Prediction: 2.9828059109437053
Item ID: 44, Prediction: 3.1731889081503266
Item ID: 686, Prediction: 3.456462778996828
Item ID: 28, Prediction: 3.6733756404861233
Item ID: 729, Prediction: 3.2916280509285403
Item ID: 52, Prediction: 3.5098079008598533
Item ID: 230, Prediction: 3.2852157522855125
Item ID: 506, Prediction: 3.548973188153095
Item ID: 566, Prediction: 3.3508811186312206
Item ID: 774, Prediction: 3.0860425603776527
Item ID: 1003, Prediction: 2.6730889697933553
Item ID: 433, Prediction: 3.3803184812318365
Item ID: 9

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
Item ID: 1663, Prediction: 3.573407369371809
Item ID: 1652, Prediction: 3.081027681564652
Item ID: 1467, Prediction: 4.2228047438456295
Item ID: 1634, Prediction: 3.5958206031634403
Item ID: 1256, Prediction: 3.3260470242935467
Item ID: 1606, Prediction: 3.5851273005982747
Item ID: 1544, Prediction: 3.562805187789102
Item ID: 1622, Prediction: 3.3254497169970487
Item ID: 1329, Prediction: 3.566443183117189
Item ID: 1494, Prediction: 3.3414971622967276
Item ID: 1672, Prediction: 3.4067893669038085
Item ID: 1673, Prediction: 3.6303113033916494
Item ID: 1382, Prediction: 3.443910918695372
Item ID: 1633, Prediction: 3.779786476149508
Item ID: 1677, Prediction: 3.734895407651383
Item ID: 1679, Prediction: 3.7249132638429816
Item ID: 1577, Prediction: 3.5327955328179352
Item ID: 1666, Prediction: 3.589127058439132
Item ID: 1356, Prediction: 3.637398973317932
Item ID: 1635, Prediction: 3.780989565481384
Item ID: 1433, Prediction: 3.4841888005

In [11]:
import numpy as np

In [12]:
class MatrixFactorization:
    def __init__(self, num_users, num_items, num_features, learning_rate, reg_param, num_iterations):
        self.num_users = num_users
        self.num_items = num_items
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.reg_param = reg_param
        self.num_iterations = num_iterations
        self.user_matrix = np.random.rand(num_users, num_features)
        self.item_matrix = np.random.rand(num_items, num_features)

    def predict_rating(self, user_id, item_id):
        return np.dot(self.user_matrix[user_id], self.item_matrix[item_id])

    def loss_function(self, user_id, item_id, true_rating):
        predicted_rating = self.predict_rating(user_id, item_id)
        error = true_rating - predicted_rating
        regularization_term = 0.5 * (np.linalg.norm(self.user_matrix[user_id]) + np.linalg.norm(self.item_matrix[item_id]))
        return 0.5 * error**2 + 0.5 * self.reg_param * regularization_term

    def gradient_descent(self, user_id, item_id, true_rating):
        error = true_rating - self.predict_rating(user_id, item_id)
        user_gradient = -error * self.item_matrix[item_id] + self.reg_param * self.user_matrix[user_id]
        item_gradient = -error * self.user_matrix[user_id] + self.reg_param * self.item_matrix[item_id]

        self.user_matrix[user_id] -= self.learning_rate * user_gradient
        self.item_matrix[item_id] -= self.learning_rate * item_gradient

    def train(self, user_ratings):
        for _ in range(self.num_iterations):
            for user_id, item_id, rating in user_ratings:
                self.gradient_descent(user_id, item_id, rating)

    def evaluate(self, test_ratings):
        total_loss = 0.0
        for user_id, item_id, true_rating in test_ratings:
            total_loss += self.loss_function(user_id, item_id, true_rating)
        return total_loss / len(test_ratings)

In [39]:
num_users = 100
num_items = 50
num_features = 10
learning_rate = 0.01
reg_param = 0.01
num_iterations = 50

mf_model = MatrixFactorization(num_users, num_items, num_features, learning_rate, reg_param, num_iterations)

In [40]:
np.random.seed(42)
train_ratings = np.random.randint(0, 6, size=(500, 3))  # припустимо, що є 500 рейтингів у тренувальному датасеті
test_ratings = np.random.randint(0, 6, size=(100, 3))   # припустимо, що є 100 рейтингів у тестовому датасеті

mf_model.train(train_ratings)
test_loss = mf_model.evaluate(test_ratings)
print(f'Test Loss: {test_loss}')

Test Loss: 1.7750906309942487
